In [1]:
import time
import requests
def requests_view(response):
    import webbrowser
    request_url = response.url
    base_url = '<head><base href="%s">'%(request_url)
    base_url = base_url.encode()
    content = response.content.replace(b"<head>",base_url)
    tem_html = open('tmp.html','wb')
    tem_html.write(content)
    tem_html.close()
    webbrowser.open_new_tab("tmp.html") #mac fail to open

In [3]:
respond = requests.get("http://sz.ganji.com/zufang/pn1/")

In [4]:
requests_view(respond)

In [5]:
from lxml import etree
#html = etree.HTML(respond.content)
html = etree.HTML(respond.content.decode())

In [6]:
path = './/*[@class="f-main-list"]//div[contains(@class,"f-list-item ershoufang-list")]'
items = html.xpath(path)

In [7]:
for i in items:
    title = i.xpath('.//dd/a/@title')
    price = i.xpath('.//dd[@class="dd-item info"]/div/span/text()')
    size = i.xpath('.//dd[@class="dd-item size"]/span/text()')
    agent = i.xpath('.//dd[4]/span/span/text()')[0].split()

In [8]:
## Create SQLsite to store data
from sqlalchemy import create_engine
engine = create_engine('sqlite:///ganji_rential.sqlite', echo = True)

from sqlalchemy import Table,Column,Integer,String,ForeignKey  
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()
DBSession = sessionmaker(bind=engine)
session = DBSession()
class rental(Base):
    __tablename__ = "rental"
    id = Column(Integer, primary_key=True)
    title = Column(String(400))
    price = Column(String(400))
    info = Column(String(400))
    agent = Column(String(400))
    
    @classmethod
    def save(cls, data):
        session.add(data)
        session.commit()
        return data.id
        
Base.metadata.create_all(engine)

2019-07-23 11:40:19,706 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-07-23 11:40:19,707 INFO sqlalchemy.engine.base.Engine ()
2019-07-23 11:40:19,709 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-07-23 11:40:19,710 INFO sqlalchemy.engine.base.Engine ()
2019-07-23 11:40:19,711 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("rental")
2019-07-23 11:40:19,712 INFO sqlalchemy.engine.base.Engine ()


In [9]:
rent = rental()
rent.title = 'title'
rent.price = 'price'
rent.info = 'info'
rent.agent = 'agent'
rent.save(rent)

2019-07-23 11:40:19,720 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-23 11:40:19,723 INFO sqlalchemy.engine.base.Engine INSERT INTO rental (title, price, info, agent) VALUES (?, ?, ?, ?)
2019-07-23 11:40:19,723 INFO sqlalchemy.engine.base.Engine ('title', 'price', 'info', 'agent')
2019-07-23 11:40:19,726 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-23 11:40:19,730 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-23 11:40:19,731 INFO sqlalchemy.engine.base.Engine SELECT rental.id AS rental_id, rental.title AS rental_title, rental.price AS rental_price, rental.info AS rental_info, rental.agent AS rental_agent 
FROM rental 
WHERE rental.id = ?
2019-07-23 11:40:19,732 INFO sqlalchemy.engine.base.Engine (1520,)


1520

In [11]:
base_url = "http://sz.ganji.com/zufang/pn{}"
headers = {
    #'Referer':,
    'User-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36'
}

def pass_data(respond):
    html = etree.HTML(respond.content.decode())
    path = './/*[@class="f-main-list"]//div[contains(@class,"f-list-item ershoufang-list")]'
    items = html.xpath(path)
    for i in items:
        title = i.xpath('.//dd/a/@title')
        price = i.xpath('.//dd[@class="dd-item info"]/div/span/text()')
        info = i.xpath('.//dd[@class="dd-item size"]/span/text()')
        agent = i.xpath('.//dd[4]/span/span/text()')[0].split()
        rent = rental()
        rent.title = " ".join(title)
        rent.price = " ".join(price)
        rent.info = " ".join(info)
        rent.agent = " ".join(agent)
        rent.save(rent)

for num in range(71):
    re_url = base_url.format(num)
    response = requests.get(re_url, headers = headers)
    if response.status_code == 403:
        for i in range(3):
            response = requests.get(re_url, headers = headers)
            if response.status_code == 200:
                break
    if response.status_code == 200:
        pass_data(respond)
    print(re_url, "status:",response.status_code)
    time.sleep(0.5)


2019-07-23 11:40:33,022 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-23 11:40:33,046 INFO sqlalchemy.engine.base.Engine INSERT INTO rental (title, price, info, agent) VALUES (?, ?, ?, ?)
2019-07-23 11:40:33,047 INFO sqlalchemy.engine.base.Engine ('龙华大浪 房东直租  32平单间  豪华装修  全新家私家电', '850 元/月', '1室0厅1卫 32㎡ 南向 豪华装修', '来自经纪人： 文辉物业 - 桂小英')
2019-07-23 11:40:33,048 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-23 11:40:33,052 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-23 11:40:33,053 INFO sqlalchemy.engine.base.Engine SELECT rental.id AS rental_id, rental.title AS rental_title, rental.price AS rental_price, rental.info AS rental_info, rental.agent AS rental_agent 
FROM rental 
WHERE rental.id = ?
2019-07-23 11:40:33,053 INFO sqlalchemy.engine.base.Engine (1766,)
2019-07-23 11:40:33,056 INFO sqlalchemy.engine.base.Engine INSERT INTO rental (title, price, info, agent) VALUES (?, ?, ?, ?)
2019-07-23 11:40:33,056 INFO sqlalchemy.engine.base.Engine ('碧桂园品牌公寓 青年白领社

2019-07-23 11:40:33,146 INFO sqlalchemy.engine.base.Engine ('4号线龙华 清湖双地铁 汇海广场旁 南北通透 采光好大单间', '1100 元/月', '1室0厅1卫 20㎡ 东南向 精装修', '来自经纪人： 独立经纪人（龙华） - 廖哲')
2019-07-23 11:40:33,149 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-23 11:40:33,152 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-23 11:40:33,153 INFO sqlalchemy.engine.base.Engine SELECT rental.id AS rental_id, rental.title AS rental_title, rental.price AS rental_price, rental.info AS rental_info, rental.agent AS rental_agent 
FROM rental 
WHERE rental.id = ?
2019-07-23 11:40:33,154 INFO sqlalchemy.engine.base.Engine (1777,)
2019-07-23 11:40:33,156 INFO sqlalchemy.engine.base.Engine INSERT INTO rental (title, price, info, agent) VALUES (?, ?, ?, ?)
2019-07-23 11:40:33,156 INFO sqlalchemy.engine.base.Engine ('11号线！看清楚！塘尾地铁口！塘尾地铁口！网红公寓1180元', '1180 元/月', '1室0厅1卫 35㎡ 南向 豪华装修', '来自经纪人： 华越物业 - 杨耀建')
2019-07-23 11:40:33,158 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-23 11:40:33,160 INFO sqlalchemy.engine.base.Engin

2019-07-23 11:40:33,259 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-23 11:40:33,260 INFO sqlalchemy.engine.base.Engine SELECT rental.id AS rental_id, rental.title AS rental_title, rental.price AS rental_price, rental.info AS rental_info, rental.agent AS rental_agent 
FROM rental 
WHERE rental.id = ?
2019-07-23 11:40:33,261 INFO sqlalchemy.engine.base.Engine (1788,)
2019-07-23 11:40:33,264 INFO sqlalchemy.engine.base.Engine INSERT INTO rental (title, price, info, agent) VALUES (?, ?, ?, ?)
2019-07-23 11:40:33,265 INFO sqlalchemy.engine.base.Engine ('微棠社区公寓 24h专属客服安全保卫 精装修拎包住居住省心', '1299 元/月', '1室0厅1卫 30㎡ 南向 精装修', '来自经纪人： 美好颂自如寓 - 愿景明创(上梅林分行) - 齐少昆')
2019-07-23 11:40:33,267 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-23 11:40:33,270 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-23 11:40:33,271 INFO sqlalchemy.engine.base.Engine SELECT rental.id AS rental_id, rental.title AS rental_title, rental.price AS rental_price, rental.info AS rental_info, rental.

2019-07-23 11:40:33,369 INFO sqlalchemy.engine.base.Engine (1799,)
2019-07-23 11:40:33,371 INFO sqlalchemy.engine.base.Engine INSERT INTO rental (title, price, info, agent) VALUES (?, ?, ?, ?)
2019-07-23 11:40:33,371 INFO sqlalchemy.engine.base.Engine ('1580秒杀固戍海滨新村精装公寓单间，直租无中介', '1580 元/月', '1室0厅1卫 28㎡ 南向 豪华装修', '来自经纪人： 招商置业 - 程常群')
2019-07-23 11:40:33,372 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-23 11:40:33,376 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-23 11:40:33,376 INFO sqlalchemy.engine.base.Engine SELECT rental.id AS rental_id, rental.title AS rental_title, rental.price AS rental_price, rental.info AS rental_info, rental.agent AS rental_agent 
FROM rental 
WHERE rental.id = ?
2019-07-23 11:40:33,377 INFO sqlalchemy.engine.base.Engine (1800,)
2019-07-23 11:40:33,380 INFO sqlalchemy.engine.base.Engine INSERT INTO rental (title, price, info, agent) VALUES (?, ?, ?, ?)
2019-07-23 11:40:33,380 INFO sqlalchemy.engine.base.Engine ('急租！急租！丹竹头单间出租！配套齐全！拎包入住！非诚勿

2019-07-23 11:40:33,477 INFO sqlalchemy.engine.base.Engine ('精装一房一厅，适合情侣居住，带家私家电，环境优雅，拎包入住', '1350 元/月', '1室0厅1卫 20㎡ 南北向 豪华装修', '来自经纪人： 蚂蚁物业 - 赖珍琼')
2019-07-23 11:40:33,478 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-23 11:40:33,482 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-23 11:40:33,483 INFO sqlalchemy.engine.base.Engine SELECT rental.id AS rental_id, rental.title AS rental_title, rental.price AS rental_price, rental.info AS rental_info, rental.agent AS rental_agent 
FROM rental 
WHERE rental.id = ?
2019-07-23 11:40:33,484 INFO sqlalchemy.engine.base.Engine (1811,)
2019-07-23 11:40:33,487 INFO sqlalchemy.engine.base.Engine INSERT INTO rental (title, price, info, agent) VALUES (?, ?, ?, ?)
2019-07-23 11:40:33,488 INFO sqlalchemy.engine.base.Engine ('深大附近精装两房，业主精装', '5800 元/月', '2室2厅0卫 38.04㎡ 南北向 简单装修', '来自经纪人： 乐有家地产 - 黄景崇')
2019-07-23 11:40:33,490 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-23 11:40:33,493 INFO sqlalchemy.engine.base.Engine BEGIN (implic

...

2019-07-23 11:41:32,440 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-23 11:41:32,446 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-23 11:41:32,449 INFO sqlalchemy.engine.base.Engine SELECT rental.id AS rental_id, rental.title AS rental_title, rental.price AS rental_price, rental.info AS rental_info, rental.agent AS rental_agent 
FROM rental 
WHERE rental.id = ?
2019-07-23 11:41:32,451 INFO sqlalchemy.engine.base.Engine (4364,)
2019-07-23 11:41:32,457 INFO sqlalchemy.engine.base.Engine INSERT INTO rental (title, price, info, agent) VALUES (?, ?, ?, ?)
2019-07-23 11:41:32,459 INFO sqlalchemy.engine.base.Engine ('!鸿达兴花园好房出租,居住舒适,干净整洁', '880 元/月', '1室1厅1卫 36㎡ 东南向 精装修', '来自经纪人： 搜鱼租房 - 搜鱼租房(桥头分行) - 万磊')
2019-07-23 11:41:32,463 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-23 11:41:32,469 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-23 11:41:32,471 INFO sqlalchemy.engine.base.Engine SELECT rental.id AS rental_id, rental.title AS rental_title, rental.price

2019-07-23 11:41:32,717 INFO sqlalchemy.engine.base.Engine (4375,)
2019-07-23 11:41:32,723 INFO sqlalchemy.engine.base.Engine INSERT INTO rental (title, price, info, agent) VALUES (?, ?, ?, ?)
2019-07-23 11:41:32,725 INFO sqlalchemy.engine.base.Engine ('旺塘八巷3号小区 1室0厅1卫', '1700 元/月', '1室0厅1卫 28㎡ 南向', '来自个人房源：郑艳萍女士')
2019-07-23 11:41:32,730 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-23 11:41:32,737 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-23 11:41:32,739 INFO sqlalchemy.engine.base.Engine SELECT rental.id AS rental_id, rental.title AS rental_title, rental.price AS rental_price, rental.info AS rental_info, rental.agent AS rental_agent 
FROM rental 
WHERE rental.id = ?
2019-07-23 11:41:32,742 INFO sqlalchemy.engine.base.Engine (4376,)
2019-07-23 11:41:32,748 INFO sqlalchemy.engine.base.Engine INSERT INTO rental (title, price, info, agent) VALUES (?, ?, ?, ?)
2019-07-23 11:41:32,750 INFO sqlalchemy.engine.base.Engine ('五和地铁站 单间一室一厅 临近华为 购物方便  家私齐全', '1500 元/月', '1室

2019-07-23 11:41:33,028 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-23 11:41:33,036 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-23 11:41:33,038 INFO sqlalchemy.engine.base.Engine SELECT rental.id AS rental_id, rental.title AS rental_title, rental.price AS rental_price, rental.info AS rental_info, rental.agent AS rental_agent 
FROM rental 
WHERE rental.id = ?
2019-07-23 11:41:33,039 INFO sqlalchemy.engine.base.Engine (4387,)
2019-07-23 11:41:33,045 INFO sqlalchemy.engine.base.Engine INSERT INTO rental (title, price, info, agent) VALUES (?, ?, ?, ?)
2019-07-23 11:41:33,047 INFO sqlalchemy.engine.base.Engine ('登良地铁口旁的两房出租', '6500 元/月', '2室1厅0卫 57.0㎡ 北向 简单装修', '来自经纪人： Q房网 - Q房网天骄华庭分行 - 聂云祥')
2019-07-23 11:41:33,049 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-23 11:41:33,059 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-23 11:41:33,061 INFO sqlalchemy.engine.base.Engine SELECT rental.id AS rental_id, rental.title AS rental_title, rental.price AS ren

2019-07-23 11:41:33,351 INFO sqlalchemy.engine.base.Engine (4398,)
2019-07-23 11:41:33,356 INFO sqlalchemy.engine.base.Engine INSERT INTO rental (title, price, info, agent) VALUES (?, ?, ?, ?)
2019-07-23 11:41:33,358 INFO sqlalchemy.engine.base.Engine ('西丽珠光站口 安静小区 龙辉花园 精装三房 采光好 随时入住', '5800 元/月', '3室2厅1卫 87.00㎡ 南向 精装修', '来自经纪人： 好房网地产 - 好房网地产(南国丽城分行) - 尹华强')
2019-07-23 11:41:33,362 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-23 11:41:33,380 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-23 11:41:33,384 INFO sqlalchemy.engine.base.Engine SELECT rental.id AS rental_id, rental.title AS rental_title, rental.price AS rental_price, rental.info AS rental_info, rental.agent AS rental_agent 
FROM rental 
WHERE rental.id = ?
2019-07-23 11:41:33,387 INFO sqlalchemy.engine.base.Engine (4399,)
2019-07-23 11:41:33,395 INFO sqlalchemy.engine.base.Engine INSERT INTO rental (title, price, info, agent) VALUES (?, ?, ?, ?)
2019-07-23 11:41:33,399 INFO sqlalchemy.engine.base.Engine ('固戍

KeyboardInterrupt: 